In [22]:
from typing import Tuple, Dict
import numpy as np
from dataclasses import dataclass
import json
from datetime import datetime

In [26]:
@dataclass
class BinomialResult:
    price: float
    delta: float
    bond_position: float
    risk_neutral_prob: float
    is_arbitrage: bool
    replication_error: float

class OnePeriodBinomialPricer:
    '''Robust one-period binomial model implementation with comprehensive validation and arbitrage detection'''

    def __init__(self,S0:float,K:float,u:float,d:float,r:float):
        self.S0 = self._validate_positive(S0,"S0")
        self.K = self._validate_positive(K,"K")
        self.u = self._validate_multiplier(u,"u")
        self.d = self._validate_multiplier(d,"d")
        self.r = self._validate_rate(r)
        self._validate_no_arbitrage()

    def _validate_positive(self,value:float,name:str) ->float:
        if value<=0:
            raise ValueError(f"{name} must be positive, got {value} ")
        return float(value)   

    def _validate_multiplier(self,value:float,name:str) ->float:
        if value<=0:
            raise ValueError(f"Multiplier {name} must be positive, got {value} ")
        return float(value) 
    
    def _validate_rate(self,r:float) ->float:
        if r<=-1:
            raise ValueError(f"Rate must be greater than -1 , got {r} ")
        return float(r)

    def _validate_no_arbitrage(self):
        '''Check no arbitrage condition with tolerance'''
        eps = 1e-10
        if not (self.d < np.exp(self.r)-eps < self.u):
            import warnings
            warnings.warn(f'Arbitrage condition violated: d={self.d}<e^r= {np.exp(self.r):.4f} < u={self.u} expected')
    
    def price_call(self) -> BinomialResult:
        '''Price european call option'''

        #Compute terminal payoffs

        Su = self.S0*self.u
        Sd = self.S0*self.d
        Cu = max(Su - self.K,0.0)
        Cd = max(Sd - self.K,0.0)

        #Risk-Neutral Probability
        q = (np.exp(self.r)-self.d) / (self.u - self.d)

        #Price and Hedge Ratio
        price = np.exp(-self.r)*(q*Cu + (1-q)*Cd)
        delta = (Cu - Cd)/(Su - Sd)
        bond  = (Su*Cd - Sd*Cu)/((Su-Sd)*np.exp(self.r))

        #Replication error analysis
        portfolio_up = delta*Su + bond*np.exp(self.r)
        portfolio_down = delta*Sd + bond*(np.exp(self.r))
        replication_error = max(abs(portfolio_up-Cu),abs(portfolio_down-Cd))

        return BinomialResult(

            price = price,
            delta = delta,
            bond_position = bond,
            risk_neutral_prob = q,
            is_arbitrage = not (0<q<1),
            replication_error = replication_error
        )    
    def price_put(self) -> BinomialResult:
        '''Price european put option using put-call parity or direct computation'''

        call_result = self.price_call()
        forward = self.S0 - self.K*np.exp(-self.r)
        put_price = call_result.price - forward

        #Compute terminal payoffs

        Su = self.S0*self.u
        Sd = self.S0*self.d
        Pu = max(self.K - Su,0.0)
        Pd = max(self.K - Sd,0.0)
        delta = (Pu - Pd)/ (Su -Sd)


        return BinomialResult(
            price = put_price,
            delta = delta,
            bond_position = call_result.bond_position + self.K*np.exp(-self.r),
            risk_neutral_prob = call_result.risk_neutral_prob,
            is_arbitrage = call_result.is_arbitrage,
            replication_error = call_result.replication_error
        ) 
        

    def to_json(self,result:BinomialResult) ->str:
        '''Serialize results with meta data'''
        output = {
            "timestamp": datetime.now().isoformat(),
            "parameters": {
                "S0" : self.S0, "K" : self.K,"u" : self.u,
                "d" : self.d, "r": self.r
            },
            "results": {
                "price": result.price,
                "delta": result.delta,
                "bond_position":result.bond_position,
                "risk_neutral_prob":result.risk_neutral_prob,
                "is_arbitrage": result.is_arbitrage,
                "replication_error":result.replication_error

            } 
        }                       

        return json.dumps(output,indent =2)

In [27]:
##Example usage
if __name__ == "__main__":
    pricer = OnePeriodBinomialPricer(S0 = 100,K=100,u=1.12,d=0.92,r=0.02)
    call_result = pricer.price_call()
    print(pricer.to_json(call_result))

{
  "timestamp": "2026-01-02T11:46:24.997054",
  "parameters": {
    "S0": 100.0,
    "K": 100.0,
    "u": 1.12,
    "d": 0.92,
    "r": 0.02
  },
  "results": {
    "price": 5.893033233467108,
    "delta": 0.6000000000000003,
    "bond_position": -54.10696676653292,
    "risk_neutral_prob": 0.5010067001337785,
    "is_arbitrage": false,
    "replication_error": 7.105427357601002e-15
  }
}
